# Competencia: Santander Customer Satisfaction

Desde los cargos de atención al cliente, hasta en los cargos directivos la satisfacción del cliente es una de las principales métricas de éxito. Los clientes insatisfechos no se mantienen por mucho tiempo, y mas aún, los clientes insatisfechos no demuestran su insatisfacción sino hasta que ya han dejado la compañia.

El Banco Santander le esta solciitando a la comunidad de Kaggle que le ayude a identificar a los clientes insatisfechos en una etapa temprana de la relación comercial. Hacerlo le permitirá a Santander tomar medidas preventivas para mejorar la satisfacción de sus clientes antes de que sea demasiado tarde.

En esta competencia usted trabajrá con cientos de características anonimizadas para predecir si un cliente está o no satisfecho con su experiencia bancaria.

## Grupo de trabajo

- John Franklin Gonzalez Gamboa - 201725981
- Jorge Eduardo Rodriguez Cardozo - 200711501
- German Augusto Carvajal Murcia - 201313516

**Nota: para proceder con este documento es pre-requisito haber ejecutado los notebooks** **`Notebook Santander - Limpieza de datos` y `Notebook Santander - Outlier` **  **. Adicionalmente, la ruta de la carpeta `Santander` debe mantenerse inalterada entre todos los documentos**

In [1]:
import os
os.chdir(os.getcwd()+'/Santander')

## Carga de la información

Se carga la base de datos resultado del proceso de detección de datos atípicos sobre la base de entrenamiento (Notebook `Notebook Santander - Outlier`) y la base de datos de predicción (`test`). Asimismo, se importa se importan las librerias básicas necesarias para realizar el estudio.

In [2]:
import pandas as pd
import numpy as np
Data_train = pd.read_csv('Data_train.csv')
import zipfile
Data_test = zipfile.ZipFile('../Santander/test.csv.zip', 'r').open('test.csv')
Data_test = pd.io.parsers.read_table(Data_test, sep=',')
Data_test = Data_test.loc[:,Data_test.columns!='ID']

## Preprocesamiento

Para iniciar, se debe recordar que durante el proceso de limpieza de datos de la base de entrenamiento se eliminaron multiples variables del conjunto inicial. En este orden de ideas, el primer paso corresponde a la selección de las caracteristicas adecuadas que corresponderán a las variables de entrada de los modelos calibrados. 

In [3]:
Data_test = Data_test.loc[:,Data_train.columns]
Data_test= Data_test.drop('TARGET', 1)

/Users/germancarvajal/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Por otro lado, verificando la presencia de valores extremos dentro de la base de datos de predicción se identifican 2148 valores que no son consistentes con la naturaleza de las variables registradas para los clientes.

In [4]:
print("Valores extremos","\t","Total de valores extremos")
print(Data_test.values.min(),"\t\t",Data_test.stack()[Data_test.stack().values==Data_test.values.min()].count())
print(Data_test.values.max(),"\t\t",Data_test.stack()[Data_test.stack().values==Data_test.values.max()].count())

Valores extremos 	 Total de valores extremos
-999999.0 		 120
9999999999.0 		 1128


Siguiendo el procedimiento de limpieza sobre la base de entrenamiento, se procede a reemplazar dichos valores extremos por valores perdidos.

In [5]:
Data_test=Data_test.replace(-999999.0,float('NaN'))
Data_test=Data_test.replace(9999999999.0,float('NaN'))

Una vez incluidos los valores perdidos dentro de la base de datos construida, a continuación, se presenta el resumen de los campos vacios que existen en la base de datos de predicción preprocesada. En total se identifican 17 variables que exhiben un comportamiento de campos vacios similar al de la base de entrenamiento: variables perdidas por pares y un numero elevado de faltantes en la caracteristica `var3`.

In [6]:
h=pd.DataFrame(Data_test.isnull().sum()).sort_values(0,ascending=False)
h[h[0]>0]

,0
delta_imp_aport_var13_1y3,377
delta_num_aport_var13_1y3,377
var3,120
delta_num_compra_var44_1y3,78
delta_imp_compra_var44_1y3,78
delta_num_aport_var17_1y3,37
delta_imp_aport_var17_1y3,37
delta_num_venta_var44_1y3,31
delta_imp_venta_var44_1y3,31
delta_num_trasp_var33_in_1y3,12


Continuando con los hallazgos del proceso de revisión de la base de datos de entrenamiento, se procederá a imputar los valores faltantes de 4 de las caracteristicas con campos vacios utilizando la información contenida en las demás variables de los clientes. Unicamente se imputan valores para las caracteristicas que siguen un aparente proceso de faltantes MAR **(Para mayor información revisar el notebook `Notebook Santander - Limpieza de datos`)**


In [7]:
var_impute = ['imp_compra_var44_ult1','imp_venta_var44_ult1','num_compra_var44_ult1','num_venta_var44_ult1','delta_num_compra_var44_1y3','delta_imp_compra_var44_1y3','delta_num_venta_var44_1y3','delta_imp_venta_var44_1y3']

In [8]:
to_impute=Data_test[var_impute]
import fancyimpute
imputed=pd.DataFrame(fancyimpute.MICE(verbose=False).complete(to_impute))
imputed.columns=var_impute
Data_test[var_impute]=imputed

/Users/germancarvajal/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Una vez imputados los valores que siguen un proceso MAR, unicamente quedan 13 caracteristicas que siguen un proceso de campos vacios missing-completely-at-random para los cuales no se puede inferrir su valor a partir de las caracteristicas observables. Consecuentemente, se decide imputar dichos valores con la mediana de las caracteristicas de la base de datos de entrenamiento. Se prefiere la mediana por sobre la media por su mayor robustez frente a la presencia de valores extremos no corregidos durante la revisión de datos atípicos.

In [9]:
h=pd.DataFrame(Data_test.isnull().sum()).sort_values(0,ascending=False)
h[h[0]>0]

,0
delta_imp_aport_var13_1y3,377
delta_num_aport_var13_1y3,377
var3,120
delta_num_aport_var17_1y3,37
delta_imp_aport_var17_1y3,37
delta_num_trasp_var33_in_1y3,12
delta_imp_trasp_var33_in_1y3,12
delta_num_reemb_var17_1y3,11
delta_imp_reemb_var17_1y3,11
delta_num_aport_var33_1y3,10


In [10]:
for i in Data_test.columns:
    Data_test[i]=Data_test[i].fillna(Data_train[i].median())

Para terminar el preprocesamiento de la base de datos de predicción, se valida que no existan valores perdidos en la base de datos final y se exporta para ser utilizada en etapas posteriores del proyecto.

In [11]:
print('Total de datos faltantes tras la eliminación: ',Data_train.isnull().sum().sum())

Total de datos faltantes tras la eliminación:  0


In [12]:
Data_test.to_csv('Data_test.csv', index=False)